In [1]:
import pandas as pd
import numpy as np
from analysis.utils import process_categorical

In [2]:
df = pd.read_csv('../data/df_clean.csv', index_col=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17796 entries, 0 to 18357
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17796 non-null  object 
 1   League                    17796 non-null  object 
 2   NationalLeagueLevel       17796 non-null  object 
 3   Club                      17796 non-null  object 
 4   No                        17511 non-null  float64
 5   Name                      17796 non-null  object 
 6   Value                     17796 non-null  float64
 7   HighestValue              17796 non-null  float64
 8   Age                       17794 non-null  float64
 9   Height                    16717 non-null  float64
 10  Nationality               17796 non-null  object 
 11  Foot                      16124 non-null  object 
 12  Position                  17796 non-null  object 
 13  Consultancy               14524 non-null  object 
 14  Suppli

In [4]:
df = df.loc[df['Position'] == 'Torwart']  # only use tw-data

In [5]:
# select the features which provide value. the feature selection is done by analyzing the 2-dimensional jointplots of the features processing notebook and by comparing the model performance by comparing the r2 on 25%-test-data split
features = ['LeagueCountry', 'NationalLeagueLevel', 'InternationalTeam', 'League', 'Club', 'Age', 'ClubSince', 'ActiveInternational', 'FormerInternational', 'InternationalGames', 'Trending', 'Games', 'PointsPerGame', 'Cards', 'Minutes', 'TwCleanSheets', 'TwGoalsConceded', 'TwPenaltySavesQuote', 'Value']

df = df[features]

In [6]:
numeric_columns = df.select_dtypes(include=[np.number])
for col in numeric_columns.columns:
    print(f"Column: {col}, NaN values: {numeric_columns[col].isna().sum()}")

Column: Age, NaN values: 0
Column: ClubSince, NaN values: 10
Column: ActiveInternational, NaN values: 0
Column: FormerInternational, NaN values: 0
Column: InternationalGames, NaN values: 863
Column: Trending, NaN values: 0
Column: Games, NaN values: 568
Column: PointsPerGame, NaN values: 568
Column: Cards, NaN values: 568
Column: Minutes, NaN values: 568
Column: TwCleanSheets, NaN values: 568
Column: TwGoalsConceded, NaN values: 568
Column: TwPenaltySavesQuote, NaN values: 287
Column: Value, NaN values: 0


In [7]:
# drop the data records which have missing data. due to the fact that we have a simple model trained on almost all the data, we can drop around a third of the data records and still having enough data to train for a more extensive model (see performance-xxx.csv)
df = df.dropna(subset=['Age', 'ClubSince', 'Games', 'PointsPerGame', 'Cards', 'Minutes', 'TwCleanSheets', 'TwGoalsConceded', 'TwPenaltySavesQuote'])

In [8]:
# fill nan values with zero if it can be assumed, that missing data equals zero data
df = df.fillna({'InternationalGames': 0})

In [9]:
numeric_columns = df.select_dtypes(include=[np.number])
for col in numeric_columns.columns:
    print(f"Column: {col}, NaN values: {numeric_columns[col].isna().sum()}")

Column: Age, NaN values: 0
Column: ClubSince, NaN values: 0
Column: ActiveInternational, NaN values: 0
Column: FormerInternational, NaN values: 0
Column: InternationalGames, NaN values: 0
Column: Trending, NaN values: 0
Column: Games, NaN values: 0
Column: PointsPerGame, NaN values: 0
Column: Cards, NaN values: 0
Column: Minutes, NaN values: 0
Column: TwCleanSheets, NaN values: 0
Column: TwGoalsConceded, NaN values: 0
Column: TwPenaltySavesQuote, NaN values: 0
Column: Value, NaN values: 0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 18333
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LeagueCountry        1459 non-null   object 
 1   NationalLeagueLevel  1459 non-null   object 
 2   InternationalTeam    826 non-null    object 
 3   League               1459 non-null   object 
 4   Club                 1459 non-null   object 
 5   Age                  1459 non-null   float64
 6   ClubSince            1459 non-null   float64
 7   ActiveInternational  1459 non-null   int64  
 8   FormerInternational  1459 non-null   int64  
 9   InternationalGames   1459 non-null   float64
 10  Trending             1459 non-null   int64  
 11  Games                1459 non-null   float64
 12  PointsPerGame        1459 non-null   float64
 13  Cards                1459 non-null   float64
 14  Minutes              1459 non-null   float64
 15  TwCleanSheets        1459 non-null   

In [11]:
# create dummy variables for nominal variables. make sure, only a dummy variable is created, if the value occurs 5 or 10 times. The other, less important, values are assigned to a 'other'-class
df = process_categorical(df, 'InternationalTeam', 5)
df = process_categorical(df, 'League')
df = process_categorical(df, 'Club')
df = process_categorical(df, 'LeagueCountry', 5)
df = process_categorical(df, 'NationalLeagueLevel', 5)

In [12]:
df.to_csv('../data/df_extensive_model_tw.csv')